<a href="https://colab.research.google.com/github/anselvelazquez/chatbot_MS/blob/main/PoC_MS_chatbout_DATUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) IMPORT PACKAGES

In [1]:
!pip install transformers>=4.32.0 optimum>=1.12.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install langchain
!pip install chromadb
!pip install sentence_transformers # ==2.2.2
!pip install unstructured
!pip install pdf2image
!pip install pdfminer.six
!pip install unstructured-pytesseract
!pip install unstructured-inference
!pip install faiss-gpu
!pip install pikepdf
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
  Using cached faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (85.5 MB)
  Using cached pikepdf-8.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
  Using cached pillow-10.3.0-cp310-cp310-manylinux_2_28_x86_64.whl (4.5 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [2]:
!pip install Pillow

In [3]:
!pip install pillow-heif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.5 MB/s eta 0:00:00


In [4]:
#import gradio as gr
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.8 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=de8e1a5fb46c657e0c71129f3bdcf437b5e2ad9b41f1170bd228748a4554e48e
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0


# 2) CARGAMOS LLAMA 2

In [23]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

#model_name = "TheBloke/Llama-2-13b-Chat-GPTQ"

model_name = "gradientai/Llama-3-8B-Instruct-262k"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 3) PROMPT TEMPLATE LLAMA 2

In [6]:
from textwrap import fill
from langchain.prompts import PromptTemplate

template = """
[INST] <>
You are an AI assistant. You are truthful, unbiased and honest in your response.

If you are unsure about an answer, truthfully say "I don't know"
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

text = "Explain artificial intelligence in a few lines"
result = llm(prompt.format(text=text))
print(fill(result.strip(), width=100))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[INST] <> You are an AI assistant. You are truthful, unbiased and honest in your response.  If you
are unsure about an answer, truthfully say "I don't know" <>  Explain artificial intelligence in a
few lines [/INST]  Sure! Here's my explanation of artificial intelligence:  Artificial intelligence
(AI) refers to the development of computer systems that can perform tasks that typically require
human intelligence, such as learning, problem-solving, and decision-making. These systems use
algorithms and machine learning techniques to analyze data and make predictions or take actions
based on that data. Some examples of AI include natural language processing, image recognition, and
autonomous vehicles. Overall, AI is a rapidly advancing field that has the potential to transform
many industries and aspects of our lives.


In [24]:
from textwrap import fill
from langchain.prompts import PromptTemplate

template = """
[INST] <>
You are an AI assistant. You are truthful, unbiased and honest in your response.

If you are unsure about an answer, truthfully say "I don't know"
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

text = "Explain artificial intelligence in a few lines"
result = llm(prompt.format(text=text))
print(fill(result.strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[INST] <> You are an AI assistant. You are truthful, unbiased and honest in your response.  If you
are unsure about an answer, truthfully say "I don't know" <>  Explain artificial intelligence in a
few lines [/INST] assistant: Artificial Intelligence (AI) is the development of computer systems
that can perform tasks that typically require human intelligence such as visual perception, speech
recognition, decision-making, and language translation. It involves the use of algorithms and
machine learning techniques to enable computers to learn from data and make decisions based on that
information.</s>


In [25]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 4) RAG CSV

In [26]:
# Export the DataFrame to a CSV file
#csv_file_path = '/content/campos_reduce_3_encode.csv'
#csv_file_path = '/content/campos_datum.csv'
csv_file_path = '/content/UUAAs_2.csv'
#df.to_csv(csv_file_path, index=False)

from langchain_community.document_loaders.csv_loader import CSVLoader

#loader = CSVLoader(file_path='/content/campos_datum.csv')
loader = CSVLoader(file_path=csv_file_path,csv_args = {
                "delimiter": ';',
#                 "quotechar": csv.Dialect.quotechar,
            })
data = loader.load()

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_csv_doc = text_splitter.split_documents(data)
len(chunked_csv_doc)

27266

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
from langchain.vectorstores import FAISS
db_pdf = FAISS.from_documents(chunked_csv_doc, embeddings)

In [30]:
%%time

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """
[INST] <>
Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.
Please, in all your answers try to put the field with its description and the table where the field is located. Always try to put all the options you find, do not keep only one.
<>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_pdf.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)
query = "Is there any field where can I show the email adress of the customer?"
result = Chain_pdf.invoke(query)
print(fill(result['result'].strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[INST] <> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer. Please, in all your
answers try to put the field with its description and the table where the field is located. Always
try to put all the options you find, do not keep only one. <>  Campo ID: 19786013 Nombre fisico del
campo: gf_email_desc Descripcion del capo: Dirección de correo electrónico de la Persona Nombre
logico del campo: DESCRIPCION DIRECCION DE E-MAIL Tipo de dato: string Key: FALSE Formato:  Formato
logico: ALPHANUMERIC(255) Mandatory: FALSE Nombre fisico de la tabla origen: t_kbtq_customer_info
Campo ID: 19785922 Nombre fisico del campo: gf_email_desc Descripcion del capo: Dirección de correo
electrónico de la Persona Nombre logico del campo: DESCRIPCION DIRECCION DE E-MAIL Tipo de dato:
string Key: FALSE Formato: 

In [ ]:
query = "In which field and which table can I query the customer id?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Based on the provided context, you can query the customer ID in the "gf_customer_id" field, which is
located in the "t_kbtq_customer_info_his" table.


In [22]:
#New context:
query = "In which field and which table can I query the customer id? What is the descriptions of these fields?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Based on the provided context, you can query the customer ID in the following fields:  1.
gf_customer_id (t_kbtq_customer_info)         * Description: Codificación interna de la Entidad para
la identificación uní­voca de cada uno de los clientes. 2. gf_customer_id
(t_kbtq_customer_counterparty)         * Description: Codificación interna de la Entidad para la
identificación uní­voca de cada uno de los clientes. 3. gf_customer_id
(t_kbtq_customer_repositories)         * Description: Codificación interna de la Entidad para la
identificación uní­voca de cada uno de los clientes. 4. gf_customer_id (t_kbtq_eom_customer_his)
* Description: Codificación interna de la Entidad para la identificación uní­voca de cada uno de los
clientes.  All of these fields have the same name and description, indicating that they are all
referring to the same entity, which is the customer ID. The difference lies in the table where they
are located.  Note that the mandatory status of each field is also provided,

In [31]:
#LLAMA 3:
query = "In which field and which table can I query the customer id? What is the descriptions of these fields?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


assistant:   The customer id can be queried in the "gf_customer_id" field of several tables
including "t_kbtq_customer_info", "t_kbtq_customer_counterparty", "t_kbtq_customer_repositories",
and "t_kbtq_eom_customer_his". The description of this field is "Codificación interna de la Entidad
para la identificación uní­voca de cada uno de los clientes." It has a data type of string and is
mandatory for some tables but not others.</s>


In [ ]:
query = "In which field and in which tables can I query the customer phone number?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Based on the provided context, you can query the customer phone number in the following fields and
tables:  1. Field: gf_extension_phone_id (IDENTIFICADOR EXTENSION TELEFONO) Table:
t_kbtq_customer_info_his  This field contains the extension number of the customer's phone number.
2. Field: gf_prefix_phone_desc (DESCRIPCION DEL PREFIJO DEL TELEFONO) Table:
t_kbtq_customer_info_his  This field contains the description of the prefix of the customer's phone
number.  3. Field: gf_prefix_phone_id (IDENTIFICADOR PREFIJO TELEFONO) Table:
t_kbtq_customer_info_his  This field contains the code of the prefix of the customer's phone number.
Note that these fields are not mandatory, so it is possible that some customers may not have a phone
number recorded in these fields.


In [33]:
#LLAMA 3:
query = "In which field and in which tables can I query the customer phone number? Tell me all the options"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


assistant:   The customer phone number can be queried in the "gf_phone_number" field of the
"t_kbtqcustomer_contact" table.</s>


In [ ]:
query = "In which field and in which tables can I query the contract id?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Based on the provided context, you can query the contract id in the following fields and tables:  1.
Field: `g_contract_id` in table `t_p6r1_prt_cont_prdt_bal_board`. 2. Field: `g_contract_id` in table
`t_kmol_transfer_movements`.  These are the only two fields with the name `g_contract_id` in the
given tables.


In [ ]:
query = "Is there a field where I can check the postal code of the beneficiary of a transfer? In what table can I find it?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Based on the provided context, the field that corresponds to the postal code of the beneficiary is
"gf_beneficiary_zipcode_id" in the "t_kmol_transfer_movements" table. This field is described as
"Identificador correspondiente al código postal del beneficiario" and its type is specified as
"string".


In [ ]:
query = "Is there a field where I can check the product family id? In what table can I find it?"
result = Chain_pdf.invoke(query)
result = result['result'].split('[/INST]')[1].strip()
print(fill(result.strip(), width=100))

Based on the provided context, the field where you can check the product family ID is
"g_global_product_family_id" and it is located in the "t_p6r1_prt_cont_prdt_bal_board" table.


In [18]:
import gradio as gr
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )

        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=db_pdf.as_retriever(),
            memory=memory,
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history}, return_only_outputs=True)
        print("Result:", result)

        chat_history.append((query, result['answer'].partition('\nHelpful Answer: ')[2]))
        return '', chat_history


    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

# build gradio ui
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='DATUM chatbot by Management Solutions')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2f0b3f403ab9c47d85.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Result: {'answer': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nCampo ID: 17665617\nNombre fisico del campo: gf_correspondence_language_id\nDescripcion del capo: Idioma en el que el cliente desea recibir la correspondencia\nNombre logico del campo: IDENTIFICADOR IDIOMA DE CORRESPONDENCIA\nTipo de dato: string\nKey: FALSE\nFormato: \nFormato logico: ALPHANUMERIC(5)\nMandatory: FALSE\nNombre fisico de la tabla origen: t_kbtq_cust_contact_chan_lcl\n\nCampo ID: 17665821\nNombre fisico del campo: gf_correspondence_language_id\nDescripcion del capo: Idioma en el que el cliente desea recibir la correspondencia\nNombre logico del campo: IDENTIFICADOR IDIOMA DE CORRESPONDENCIA\nTipo de dato: string\nKey: FALSE\nFormato: \nFormato logico: ALPHANUMERIC(5)\nMandatory: FALSE\nNombre fisico de la tabla origen: t_kbtq_cust_contact_chan_his\n\nCampo ID: 17657913\nNombre fisico del 